In [4]:
import numpy as np
import pandas as pd

In [2]:
import h2o

h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.391-b13, mixed mode)
  Starting server from C:\Users\33549\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\33549\AppData\Local\Temp\tmptjp4qg_f
  JVM stdout: C:\Users\33549\AppData\Local\Temp\tmptjp4qg_f\h2o_33549_started_from_python.out
  JVM stderr: C:\Users\33549\AppData\Local\Temp\tmptjp4qg_f\h2o_33549_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,"21 days, 14 hours and 22 minutes"
H2O_cluster_name:,H2O_from_python_33549_964y41
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.493 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [5]:
data = pd.read_csv('TP bal.csv',skiprows=[0]) 
data = data.dropna(axis=1, how='all')
data = data.drop(columns=['Sample_ID'])

# 将所有非数值型的数据替换为 NaN
data.replace('-', np.nan, inplace=True)

# 将每列的 NaN 值用该列的平均值填充
data = data.apply(lambda col: col.fillna(col.mean()))

,SiO2,TiO2,Al2O3,FeOt,MnO,MgO,CaO,Na2O,K2O,Cr2O3,...,Al2O3.1,FeOt.1,MnO.1,MgO.1,CaO.1,Na2O.1,K2O.1,Cr2O3.1,P_GPa,T_K
0,50.10,1.54,14.78,11.35,0.230000,7.52,11.49,2.40,0.090000,0.105317,...,6.72,7.21,0.1900,15.55,17.27,0.61,0.010000,0.400047,0.2047,1448.15
1,50.40,1.19,15.76,9.52,0.180000,7.98,12.20,2.30,0.070000,0.105317,...,5.31,7.68,0.1500,16.36,16.34,0.81,0.040000,0.400047,0.2037,1443.15
2,59.90,0.70,19.00,2.96,0.210000,0.32,0.66,10.60,5.700000,0.000000,...,2.25,12.90,1.0400,10.10,20.40,1.66,0.000000,0.000000,0.2000,1123.15
3,59.60,0.60,19.00,3.11,0.220000,0.29,0.68,10.80,5.700000,0.000000,...,1.65,13.10,1.1100,10.70,21.30,0.71,0.000000,0.000000,0.2000,1123.15
4,61.50,0.31,19.40,2.67,0.150000,0.18,0.45,10.10,5.210000,0.000000,...,2.13,14.90,1.3000,7.20,19.90,1.70,0.300000,0.000000,0.2000,1073.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,50.40,2.10,13.50,11.70,0.220000,7.05,11.50,2.66,0.170000,0.105317,...,2.47,6.59,0.1700,16.90,19.90,0.21,0.078016,0.400047,0.0001,1444.15
965,50.10,2.52,13.40,11.90,0.160000,6.30,10.80,3.31,0.380000,0.105317,...,2.84,7.29,0.1300,15.70,20.60,0.28,0.078016,0.400047,0.0001,1425.15
966,49.20,4.17,12.00,14.30,0.300000,5.51,9.90,2.68,0.320000,0.105317,...,3.14,9.70,0.2400,15.20,18.00,0.31,0.078016,0.400047,0.0001,1401.15
967,50.78,1.93,16.44,9.79,0.160021,6.87,10.59,2.98,1.899743,0.105317,...,11.83,6.01,0.2351,12.89,16.67,2.02,0.078016,0.400047,3.0000,1673.15


In [14]:
from h2o.frame import H2OFrame
h2o_data = H2OFrame(data)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [15]:
train, valid, test = h2o_data.split_frame(ratios=[0.7, 0.15], seed=42)

In [20]:
x = h2o_data.columns[:-2]
y_temp = 'T_K'
y_pressure = 'P_GPa'

In [22]:
from h2o.automl import H2OAutoML

# Set up AutoML
aml_temp = H2OAutoML(max_models=10, seed=42, stopping_metric='RMSE', sort_metric='RMSE')
aml_pressure = H2OAutoML(max_models=10, seed=42, stopping_metric='RMSE', sort_metric='RMSE')

# 训练模型
aml_temp.train(x=x, y=y_temp, training_frame=train, validation_frame=valid)
aml_pressure.train(x=x, y=y_pressure, training_frame=train, validation_frame=valid)

AutoML progress: |
00:26:53.651: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
00:26:53.657: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
00:27:02.382: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.
00:27:02.382: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/10
# GBM base models (used / total),5/6
# DRF base models (used / total),1/2
# DeepLearning base models (used / total),1/1
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [23]:
lb_temp = aml_temp.leaderboard
lb_pressure = aml_pressure.leaderboard

print("Temperature Leaderboard:")
print(lb_temp)

print("\nPressure Leaderboard:")
print(lb_pressure)

Temperature Leaderboard:
model_id                                                   rmse      mse      mae      rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20231130_02653     40.2215  1617.77  28.2161  0.0283959                   1617.77
StackedEnsemble_BestOfFamily_1_AutoML_1_20231130_02653  40.679   1654.78  28.4786  0.0287263                   1654.78
GBM_2_AutoML_1_20231130_02653                           41.0484  1684.97  28.6179  0.0289844                   1684.97
GBM_4_AutoML_1_20231130_02653                           41.275   1703.63  29.1914  0.0291495                   1703.63
GBM_5_AutoML_1_20231130_02653                           42.119   1774.01  29.5553  0.0297335                   1774.01
GBM_grid_1_AutoML_1_20231130_02653_model_1              42.4252  1799.9   30.25    0.0300793                   1799.9
GBM_3_AutoML_1_20231130_02653                           42.4865  1805.1   30.1449  0.0300843                   1805.1
XRT_1_AutoML_1_20231130_0

In [24]:
pred_temp = aml_temp.predict(test)
pred_pressure = aml_pressure.predict(test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
